In [ ]:
'''
Based On
URL: https://github.com/PKU-RL/DGN
'''

# Import Libraries

In [1]:
from collections import deque
import random
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import os
import copy
import sys
import time
from tqdm import tqdm
import psutil

import torch
import torch.nn as nn
from torch import optim


pygame 2.5.2 (SDL 2.28.2, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


'\nimport tensorflow as tf\n\nimport os, sys, time\nfrom keras import backend as K\nfrom keras.optimizers import Adam\nimport tensorflow as tf\nimport random\nfrom ReplayBuffer_v2 import ReplayBuffer\nfrom keras.layers import Activation, Dense, Dropout, Conv2D, Input, Lambda, Flatten, TimeDistributed\nfrom keras.layers import Add, Reshape, MaxPooling2D, Concatenate, Embedding, RepeatVector\nfrom keras.models import Model\n\n#from keras.utils import np_utils,to_categorical\nfrom keras.utils import to_categorical\n#from keras.engine.topology import Layer\nfrom tensorflow.keras.layers import Layer\n'

In [3]:
%run src/IotNodes.ipynb
%run src/BaseStation.ipynb
%run src/Agent.ipynb
%run src/Block.ipynb

In [4]:
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

# Buffer

In [5]:
class ReplayBuffer(object):

    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.buffer = deque(maxlen=self.buffer_size)

    def getBatch(self, batch_size):
        sample_size = min(batch_size, len(self.buffer))
        return random.sample(self.buffer, sample_size)


    def size(self):
        return self.buffer_size

    def add(self, state, action, reward, new_state, done, adjacency):
        experience = (state, action, reward, new_state, done, adjacency)
        self.buffer.append(experience)

    def count(self):
        return len(self.buffer)

    def erase(self):
        self.buffer = deque(maxlen=self.buffer_size)

# Graph

In [6]:
class Graph():

    def __init__(self, n, m, k, neighbors, iot_rate, def_ttl, epsilon):    
        self.row = n
        self.col = m
        self.k = k
        self.neighbors = neighbors
        self.action_space = self.neighbors - self.k
        self.len_feature =  2*self.action_space + 2 
        self.epsilon = epsilon

        self.node_sq_dist = {}
        self.node_to_matrix_dict = {node: [node//self.col, node%self.col] for node in range(self.row*self.col)}
        self.matrix_to_node_dict = {(node//self.col, node%self.col): node for node in range(self.row*self.col)}
        
        self.Agents = []
        self.IotNodes = []
        self.BaseStation = None
        
        self.G = nx.Graph()
        
        self.defTtl = def_ttl
        self.iot_rate = iot_rate
        self.n_nodes = self.row*self.col

In [7]:
class Graph(Graph):
    
    def compute_sq_dist(self, node, base_node):
        r, c = self.node_to_matrix_dict[node]
        b_r, b_c = self.node_to_matrix_dict[base_node]

        sq_dist = (b_r-r)**2 + (b_c-c)**2
        
        return sq_dist

    def create_sq_dist_dict(self):
        base_node = self.map_info_dict['BaseStation'][0]
        self.node_sq_dist = {node: self.compute_sq_dist(node, base_node) for node in range(self.row*self.col)}
        
        return

In [8]:
class Graph(Graph):

    def read(self, Map):
        self.G.add_nodes_from(Map['nodes'])
        self.G.add_edges_from(Map['edges'])

        for node in Map['BaseStation']:
            x,y = self.node_to_matrix_dict[node]
            self.BaseStation = BaseStation(x,y)
            self.G.nodes[node]["obj"] = self.BaseStation

        self.create_sq_dist_dict()

        for node in Map['Agents']:
            x,y = self.node_to_matrix_dict[node]
        
            node_neighbours = [edge[1] for edge in self.G.edges(node) if ((edge[1] in Map['Agents']) or (edge[1] in Map['BaseStation']))]

            neigh_sq_dist = {key: self.node_sq_dist[key] for key in node_neighbours}
   
            sorted_neigh_sq_dist = sorted(neigh_sq_dist.items(), key=lambda x: x[1])
            reduce_neighbours = dict(sorted_neigh_sq_dist[:self.action_space])

            agent = Agent(node, list(reduce_neighbours.keys()), x, y, self.BaseStation)
            self.G.nodes[node]["obj"] = agent
            self.Agents.append(agent)

        for node in Map['IotNodes']:
            x,y = self.node_to_matrix_dict[node]
            iot_neighbours = [edge[1] for edge in self.G.edges(node) if ((edge[1] in Map['Agents']) or (edge[1] in Map['BaseStation']))]
            
            iot = IotNodes(self.iot_rate, self.defTtl,x,y)
        
            for neigh in iot_neighbours:
                iot.addNeighbour(neigh)
            
            self.G.nodes[node]["obj"] = iot
            self.IotNodes.append(iot)
        
        for node in Map['Blocks']:
            x,y = self.node_to_matrix_dict[node]
            
            block= Block((x,y))
            self.G.nodes[node]["obj"] = block

In [10]:
class Graph(Graph):

    def from_file_graph(self, map_name):
        with open(map_name, 'rb') as file:
            self.map_info_dict = pickle.load(file)
        self.read(self.map_info_dict)


In [11]:
class Graph(Graph):
    def getBaseStation(self):
        return self.BaseStation

    def getAgents(self):
        return self.Agents
  
    def getIotNodes(self):
        return self.IotNodes 

In [12]:
class Graph(Graph):
    def renderMap(self):
        print('-'*6*self.col,end="")
        print()
        for r in range(self.row):
            for c in range(self.col):
                node = self.matrix_to_node_dict[(r,c)]
                object = self.G.nodes[node]["obj"]
                if object.isBlock():
                    print('|',end="")
                    print('X'.ljust(5),end="")
                elif object.isBase():
                    print('|',end="")
                    cell = 'B' + str(object.getVal())
                    print(cell.ljust(5),end="")
                elif object.isIot():
                    print('|',end="")
                    cell = 'I' + str(object.queue)
                    print(cell.ljust(5),end="")
                else:
                    print('|',end="")
                    cell = 'A' + str(object.getVal())
                    print(cell.ljust(5),end="")
            print('|')
            print('-'*6*self.col,end="")
            print()
        print()
        return

In [13]:
class Graph(Graph):
    def str_renderMap(self):
        str_map = ""
        str_map += '-'*6*self.col
        str_map += '\n'
        
        for r in range(self.row):
            for c in range(self.col):
                node = self.matrix_to_node_dict[(r,c)]
                object = self.G.nodes[node]["obj"]
                if object.isBlock():
                    str_map += '|'
                    str_map += 'X'.ljust(5)
                elif object.isBase():
                    str_map += '|'
                    cell = 'B' + str(object.getVal())
                    str_map += cell.ljust(5)
                elif object.isIot():
                    str_map += '|'
                    cell = 'I' + str(object.queue)
                    str_map += cell.ljust(5)
                else:
                    str_map += '|'
                    cell = 'A' + str(object.getVal())
                    str_map += cell.ljust(5)
            str_map += '|' + '\n'
            str_map += '-'*6*self.col + '\n'
        str_map += '\n'
        return str_map

In [15]:
class Graph(Graph):
    def reset(self):
        self.BaseStation.reset()

        for agent in self.Agents:
            agent.reset()
            
        for iot in self.IotNodes:
            iot.reset()
            
        return

In [16]:
class Graph(Graph):
    # Base should be -1 because it is always the best option.
    def obtain_packet_queue(self, node):
        agent = self.G.nodes[node]["obj"]
        if agent.isBase() ==  True:
            return -1
        else:
            return agent.getVal() + len(agent.latest_queue)

In [21]:
class Graph(Graph):

    def perform_action(self, node, action_tensor):
        agent = self.G.nodes[node]["obj"] 

        action = 0

        for packet in agent.queue:
            packet.decrease_ttl()
            
        if agent.queue == []:
            reward = 0
            done = False
            return action, reward, done
            
        topPacket = agent.queue[-1]
        
        if topPacket.get_ttl() <= 0:
            
            expired_count = 0
            
            for packet in agent.queue:
                if packet.get_ttl() <= 0:
                    expired_count += 1

            for i in range(expired_count):
                agent.popQueue()
            agent.popQueue()
            reward = -50*expired_count
            done = False
            return action, reward, done
        
        # select the action
        neigh = agent.neighbours
        
        padded_neigh = neigh + [-1] * (self.action_space - len(neigh))
        padded_neigh = torch.tensor(padded_neigh)
        
        action_tensor[padded_neigh == -1] = 0 

        # prevents looping
        for ind in range(len(action_tensor)):
            if padded_neigh[ind] in topPacket.path:
                action_tensor[ind] = 0
        
        if random.random() < self.epsilon:
            action = random.randrange(len(neigh))
        else:
            action = torch.argmax(action_tensor)

        # send the packet.

        base_node = self.map_info_dict['BaseStation'][0]

        if (base_node in agent.neighbours):
            topPacket = agent.popQueue()
            self.BaseStation.acceptPacket(topPacket)

            reward = 10
            done = True
            return action, reward, done
            
        receiver_node = agent.neighbours[action]
        receiver = self.G.nodes[receiver_node]["obj"]

        topPacket = agent.popQueue()
        receiver.acceptPacket(topPacket)
        
        reward = 0
        done = False
    
        return action, reward, done
        

In [24]:
class Graph(Graph):

    def set_action(self, action_matrix):

        reward_list = []
        done_list = []
        action_list = []
        
        for node in range(self.n_nodes):
            
            if node not in self.map_info_dict['Agents']:
                
                action = -1
                reward = 0
                done = False
                
                action_list.append(action)
                reward_list.append(reward)
                done_list.append(done)
                
                continue
            action_tensor = action_matrix[node][0].float()
            

            action, reward, done = self.perform_action(node, action_tensor)

            action_list.append(action)
            reward_list.append(reward)
            done_list.append(done)

        for node in range(self.n_nodes):
            if node not in self.map_info_dict['Agents']:
                continue
            agent = self.G.nodes[node]["obj"]
            agent.update_state()  

        return action_list, reward_list, done_list

In [27]:
class Graph(Graph):
    def find_min_neighbour_node(self, iot):
        neighbours = iot.neighbours
        if neighbours == []:
            return None
        neigh_packets = {}
        for neigh_node in neighbours:
            neigh_packets[neigh_node] = self.obtain_packet_queue(neigh_node)
        min_neigh_node = min(neigh_packets, key=neigh_packets.get)

        return min_neigh_node

In [37]:
class Graph(Graph):
    def iot_send_packets(self):
        tot_packets = 0

        for iot in self.IotNodes:
            iot.generatePacket()
            tot_packets += iot.rate
            num_packets = len(iot.queue)
            
            for i in range(num_packets):
                if iot.getQueueSize() > 0:   
                    top_packet = iot.queue.pop()
                    min_neigh_node = self.find_min_neighbour_node(iot)
                    if min_neigh_node == None:
                        continue
                    neighbour = self.G.nodes[min_neigh_node]["obj"]
                    neighbour.acceptPacket(top_packet)
        
        for node in range(self.n_nodes):
            if node not in self.map_info_dict['Agents']:
                continue
            agent = self.G.nodes[node]["obj"] 
            agent.update_state()  
            
        return tot_packets

In [38]:
class Graph(Graph):
    def meanTtl(self):
        packets = self.BaseStation.packets_received
        if len(packets)==0:
            return -1
        return sum([r_packet.get_ttl() for r_packet in packets])/len(packets)

In [44]:
class Graph(Graph):
    
    def observation(self):
        obs = []
        r,c = self.BaseStation.getPosition()
        base_node = self.matrix_to_node_dict[(r,c)]
            
        for node in range(self.n_nodes): 
            
            if node not in self.map_info_dict['Agents']:
                ob = [-1]*(2*self.action_space + 2) 
                obs.append(ob) 
                continue
                
            ob = []
            
            agent = self.G.nodes[node]["obj"]
            num_packets = self.obtain_packet_queue(node)

            ob.append(num_packets)
            ob.append(-self.node_sq_dist[node])

            num_neigh = len(agent.neighbours)
            
            for i in range(num_neigh):
                neigh = agent.neighbours[i]
                neigh_num_packets = self.obtain_packet_queue(neigh)
                ob.append(neigh_num_packets)
                ob.append(-self.node_sq_dist[neigh])
                
            for j in range(self.action_space-num_neigh):
                ob.append(-1)
                ob.append(-1)
                
            obs.append(ob)
        return obs

In [47]:
class Graph(Graph):
    def Adjacency(self):
        
        adj = []
        no_neighbor_value = self.row*self.col

        for node in range(self.n_nodes):

            if node not in self.map_info_dict['Agents']:
                l = torch.zeros(self.action_space,self.n_nodes)
                adj.append(l)

                continue
            
            neigh = self.G.nodes[node]["obj"].neighbours
            padded_neigh = neigh + [-1] * (self.action_space - len(neigh))
            padded_neigh = torch.tensor(padded_neigh)
            
            padded_neigh[padded_neigh == -1] = no_neighbor_value

            l = torch.nn.functional.one_hot(padded_neigh, num_classes=self.row*self.col + 1).float()
            
            l = l[:, :-1]

            adj.append(l)

        adj = torch.stack(adj)
        adj = adj.tolist()
        return adj

# Model

In [66]:
class MLP(nn.Module):
    def __init__(self, len_feature, d=128):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(len_feature, d)
        self.fc2 = nn.Linear(d, d)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x


In [67]:
class MultiHeadsAttModel(nn.Module):
    def __init__(self, d=128, dv=16, dout=128, nv=8):
        super(MultiHeadsAttModel, self).__init__()

        self.d = d
        self.dv = dv
        self.dout = dout
        self.nv = nv
        self.device = torch.device( "cpu")

        self.v_linear = nn.Linear(self.d, self.dv * self.nv)
        self.q_linear = nn.Linear(self.d, self.dv * self.nv)
        self.k_linear = nn.Linear(self.d, self.dv * self.nv)
        self.out_linear = nn.Linear(self.dv * self.nv, self.dout)

    def forward(self, v1, q1, k1, ve):        
        
        v2 = torch.relu(self.v_linear(v1))
        q2 = torch.relu(self.q_linear(q1))
        k2 = torch.relu(self.k_linear(k1))

        batch_size = v2.shape[0] 
        
        v = v2.view(batch_size, -1, self.nv, self.dv).permute(0, 2, 1, 3)
        q = q2.view(batch_size, -1, self.nv, self.dv).permute(0, 2, 1, 3)
        k = k2.view(batch_size, -1, self.nv, self.dv).permute(0, 2, 1, 3)

        att = torch.matmul(q, k.permute(0, 1, 3, 2)) / torch.sqrt(torch.tensor([self.dv]).float().to(self.device))
        att = torch.softmax(att, dim=-1)
        out = torch.matmul(att, v)
        out = out.permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.nv * self.dv)

        T = torch.matmul(ve, out)

        out = torch.relu(self.out_linear(T))
        out = out.view(out.size(0), -1)

        return out


In [68]:
class Q_Net(nn.Module):
    def __init__(self, action_dim, d=128):
        super(Q_Net, self).__init__()

        self.concat = nn.Linear(3 * d, d)
        self.output_layer = nn.Linear(d, action_dim)
        self.softmax_layer = nn.Softmax(dim=1)

    def forward(self, I1, I2, I3):

        h_concat = torch.cat((I1, I2, I3), dim=1)
        h = torch.relu(self.concat(h_concat))
        V = self.output_layer(h)
        V_soft = self.softmax_layer(V).clone()

        return V_soft

In [70]:
######build the model#########
class Final_Model(nn.Module):
    def __init__(self, n_nodes, len_feature, action_space):
        super(Final_Model, self).__init__()
        self.encoder = MLP(len_feature)
        self.m1 = MultiHeadsAttModel()
        self.m2 = MultiHeadsAttModel()
        self.q_net = Q_Net(action_dim = action_space)
        self.n_nodes = n_nodes

    def forward(self, In):
        feature = []
        for j in range(self.n_nodes):
            feature.append(self.encoder(In[j*2]))
        
        feature_ = torch.stack(feature, dim = 1) 
        ve = In[self.n_nodes*2] 
        
        relation1 = []
        for j in range(self.n_nodes):
            T = torch.matmul(In[j*2+1], feature_)
            relation1.append(self.m1(T,T,T,ve))

        relation1_ = torch.stack(relation1, dim = 1)

        relation2 = []
        for j in range(self.n_nodes):
            T = torch.matmul(In[j*2+1], relation1_)
            relation2.append(self.m2(T,T,T,ve))
            
        V = []
        for j in range(self.n_nodes):
            V.append(self.q_net(feature[j], relation1[j], relation2[j]))

        return V


# Training and Evaluation

In [74]:
#########playing#########

def Training(graph_obj, main_model, target_model, optimizer, criterion, device, buff, batch_size, file, log_file_eval, folder_name, num_of_episodes, num_steps, iot_generate_time, create_packets_till, steps_before_train=100, tau=0.01, gamma=0.98, EP_NUM=0, old_packets=0):

    vec = torch.zeros((1,graph_obj.action_space)).tolist()
    vec[0][0] = 1

    num_packets_received = [] 
    tot_packets_sent = [] 
    mean_ttl_received = [] 
    rec_rate = []

    epsilon = graph_obj.epsilon 
        
    for ep_num in range(num_of_episodes):

        main_model.train()
        graph_obj.epsilon = epsilon
        
        loss, score = 0,0
        tot_packets = 0
        graph_obj.reset()

        old_packets_rec = 0
        old_packets_in_network = 0
        rate_per_100_steps = {}
        
        for step in range(num_steps):

            if (step % 100 == 0) or (step == num_steps - 1):
                optimizer.step()
                optimizer.zero_grad()
                
                print('ep_num ', ep_num)
                file.write('ep_num ' + str(ep_num) + '\t')
            
                print('step ', step)
                file.write('step ' + str(step) + '\n')
                
                graph_obj.renderMap()
                map_str = graph_obj.str_renderMap()
                file.write('Map \n' + map_str + '\n')
                
                print('mean score/reward ', score/(step+1) ,end='\t')
                file.write('mean score/reward ' + str(score/(step+1)) + '\t')
                
                print('total packets received ', graph_obj.BaseStation.getVal() ,end='\t')
                file.write('total packets received ' + str(graph_obj.BaseStation.getVal()) + '\t')
                
                print('total packets sent ', tot_packets, end='\t')
                file.write('total packets sent ' + str(tot_packets) + '\t')
                
                print('mean loss ', loss/(step+1))
                file.write('mean loss ' + '\t' + str(loss/(step+1)) + '\n')
                file.write('################################################################# \n\n\n')

                new_packets_rec = graph_obj.BaseStation.getVal()
                
                total_packets_received_100_hops = new_packets_rec - old_packets_rec
                total_packets_rec_per_hop = total_packets_received_100_hops/100

                receiving_rate_info = {}
                receiving_rate_info['total_packets_rec_per_hop'] = total_packets_rec_per_hop
                receiving_rate_info['old_packets_in_network'] = old_packets_in_network
                rate_per_100_steps[step] = receiving_rate_info
                
                old_packets_in_network = tot_packets - graph_obj.BaseStation.getVal()
                old_packets_rec = new_packets_rec
            
                #########evalutaing main model#########
                num_packets_received, tot_packets_received, mean_ttl_received, rec_rate = Evaluation(copy.deepcopy(graph_obj), main_model, device, log_file_eval, 1, num_steps, iot_generate_time, create_packets_till)
                eval_packets = num_packets_received[0]
                        
                if eval_packets > old_packets:
                    old_packets = eval_packets
                    torch.save(main_model.state_dict(), folder_name + '/results_GNN_10_10_reduce_' + str(graph_obj.k) + '_step_num_' + str(step) + '.pth')
            
            if(step % iot_generate_time == 0) and (step < create_packets_till):
                tot_packets += graph_obj.iot_send_packets()

            obs = graph_obj.observation()
            adj = graph_obj.Adjacency()
            ob = []
            for j in range(graph_obj.n_nodes):
                ob.append(torch.tensor([obs[j]], dtype = torch.float32).to(device)) 
                ob.append(torch.tensor([adj[j]], dtype = torch.float32).to(device)) 
            ob.append(torch.tensor([vec], dtype = torch.float32).to(device))  

            action = main_model(ob)
                    
            act, reward, done = graph_obj.set_action(action)
            next_obs = graph_obj.observation()
            
            buff.add(obs, act, next_obs, reward, done, adj)
            
            score += sum(reward)
            
            if (step < steps_before_train) and (ep_num == 0):
                continue
                
            #########training#########
            batch = buff.getBatch(batch_size)
            states, actions, new_states, rewards, dones, adj = [], [], [], [], [], []
            
            for i_ in range(graph_obj.n_nodes*2+1):
                states.append([])
                new_states.append([])
                
            for e in batch:
                for j in range(graph_obj.n_nodes):
                    states[j*2].append(e[0][j])
                    states[j*2+1].append(e[5][j])
                    new_states[j*2].append(e[2][j])
                    new_states[j*2+1].append(e[5][j])

                states[graph_obj.n_nodes*2].append(vec)
                new_states[graph_obj.n_nodes*2].append(vec)
                
                actions.append(e[1])
                rewards.append(e[3])
                dones.append(e[4])

            for j in range(graph_obj.n_nodes):
                states[j*2] = torch.tensor(states[j*2], dtype = torch.float32).to(device)
                states[j*2+1] = torch.tensor(states[j*2+1], dtype = torch.float32).to(device)
                new_states[j*2] = torch.tensor(new_states[j*2], dtype = torch.float32).to(device)
                new_states[j*2+1] = torch.tensor(new_states[j*2+1], dtype = torch.float32).to(device)

            states[graph_obj.n_nodes*2] = torch.tensor(states[graph_obj.n_nodes*2], dtype = torch.float32).to(device)
            new_states[graph_obj.n_nodes*2] = torch.tensor(new_states[graph_obj.n_nodes*2], dtype = torch.float32).to(device)
                
            actions = torch.tensor(actions) 
            rewards = torch.tensor(rewards)
            dones = torch.tensor(dones) 
                
            q_values = main_model(states) 
            target_q_values = target_model(new_states)
            
            for k in range(len(batch)):
                for j in range(graph_obj.n_nodes):
                    if dones[k][j]:
                        q_values[j][k][actions[k][j]] = rewards[k][j]
                    else:
                        q_values[j][k][actions[k][j]] = rewards[k][j] + gamma*torch.max(target_q_values[j][k])
                        
            
            main_model.train()  
            outputs = main_model(states)
            train_loss = criterion(torch.stack(outputs).to(device), torch.stack(q_values).to(device))
            train_loss.backward()
                    
            loss+=train_loss.item()
            
            #########training target model#########
            
            for param, target_param in zip(main_model.parameters(), target_model.parameters()):
                target_param.data = tau * param.data + (1 - tau) * target_param.data
                    
        num_packets_received.append(graph_obj.BaseStation.getVal())
        tot_packets_sent.append(tot_packets)
        mean_ttl_received.append(graph_obj.meanTtl())
                
        num_packets_received, tot_packets_received, mean_ttl_received, rec_rate = Evaluation(graph_obj, main_model, device, log_file_eval, 1, num_steps, iot_generate_time, create_packets_till)
        eval_packets = num_packets_received[0]

        torch.save(main_model.state_dict(), folder_name + '/train/main_model_results_GNN_10_10_reduce_' + str(graph_obj.k) + '_ep_num_' + str(EP_NUM) + '.pth')
        torch.save(target_model.state_dict(), folder_name + '/train/target_model_results_GNN_10_10_reduce_' + str(graph_obj.k) + '_ep_num_' + str(EP_NUM) + '.pth')
        
        if eval_packets > old_packets:
            old_packets = eval_packets
            torch.save(main_model.state_dict(), folder_name + '/results_GNN_10_10_reduce_' + str(graph_obj.k) + '_ep_num_' + str(EP_NUM) + '.pth')
            
    return num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets

In [75]:
def Evaluation(graph_obj, given_model, device, file, num_of_eval_episodes, num_steps, iot_generate_time, create_packets_till):


    vec = torch.zeros((1,graph_obj.action_space)).tolist() 
    vec[0][0] = 1
    
    rewards_eval = [] 
    num_packets_received = [] 
    tot_packets_sent = [] 
    mean_ttl_received = [] 
    rec_rate = []

    given_model.eval()
    graph_obj.epsilon = 0

    for ep_num in range(num_of_eval_episodes):
        
        loss, score = 0,0
        tot_packets = 0
        graph_obj.reset()

        old_packets_rec = 0
        old_packets_in_network = 0
        rate_per_100_steps = {}

        for step in tqdm(range(num_steps), desc='Episode ' + str(ep_num) + ', Number of Steps ', unit='step'):
            if(step % iot_generate_time == 0) and (step < create_packets_till):
                tot_packets += graph_obj.iot_send_packets()
                
            obs = graph_obj.observation()
            adj = graph_obj.Adjacency()
            ob = []
            
            for j in range(graph_obj.n_nodes):
                ob.append(torch.tensor([obs[j]], dtype = torch.float32).to(device)) 
                ob.append(torch.tensor([adj[j]], dtype = torch.float32).to(device)) 
            ob.append(torch.tensor([vec], dtype = torch.float32).to(device))
            
            action = given_model(ob)
                    
            act, reward, done = graph_obj.set_action(action)
            next_obs = graph_obj.observation()
            
            score += sum(reward)
            
            if (step % 100 == 0) or (step == num_steps - 1):
                file.write('ep_num ' + str(ep_num) + '\t')
            
                file.write('step ' + str(step) + '\n')
                
                map_str = graph_obj.str_renderMap()
                file.write('Map \n' + map_str + '\n')
                
                file.write('mean score/reward ' + str(score/(step+1)) + '\t')
                
                file.write('total packets received ' + str(graph_obj.BaseStation.getVal()) + '\t')
                
                file.write('total packets sent ' + str(tot_packets) + '\t')
                
                file.write('mean loss ' + '\t' + str(loss/(step+1)) + '\n')
                file.write('################################################################# \n\n\n')

                new_packets_rec = graph_obj.BaseStation.getVal()
                
                total_packets_received_100_hops = new_packets_rec - old_packets_rec
                total_packets_rec_per_hop = total_packets_received_100_hops/100

                receiving_rate_info = {}
                receiving_rate_info['total_packets_rec_per_hop'] = total_packets_rec_per_hop
                receiving_rate_info['old_packets_in_network'] = old_packets_in_network
                rate_per_100_steps[step] = receiving_rate_info
                
                old_packets_in_network = tot_packets - graph_obj.BaseStation.getVal()
                old_packets_rec = new_packets_rec
    
        num_packets_received.append(graph_obj.BaseStation.getVal())
        tot_packets_sent.append(tot_packets)
        mean_ttl_received.append(graph_obj.meanTtl())
        rec_rate.append(rate_per_100_steps)

    return num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate

# Param Setting

## k = 0

In [76]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 0 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 
epsilon = 0.3 

capacity = 200000
buff_k_0 = ReplayBuffer(capacity)

graph_k_0 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_0.from_file_graph(map_name)
graph_k_0.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_0 = Final_Model(graph_k_0.n_nodes, graph_k_0.len_feature, graph_k_0.action_space) # model
model_t_k_0 = Final_Model(graph_k_0.n_nodes, graph_k_0.len_feature, graph_k_0.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_0.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_0 = model_k_0.to(device)
model_t_k_0 = model_t_k_0.to(device)

print(model_k_0) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_0.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_0.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_0) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_0, model_k_0, model_t_k_0, optimizer, criterion, device, buff_k_0, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:53<00:00, 17.64step/s]


ep_num  0
step  100
------------------------------------------------------------
|A0   |X    |X    |A0   |A39  |I[]  |A37  |A1   |X    |X    |
------------------------------------------------------------
|A1   |A1   |X    |A42  |X    |A48  |A45  |X    |A19  |A0   |
------------------------------------------------------------
|A0   |A43  |A15  |A9   |X    |A34  |A10  |A0   |A24  |A16  |
------------------------------------------------------------
|A12  |A11  |A11  |I[]  |A9   |A43  |A3   |A15  |A16  |A9   |
------------------------------------------------------------
|A1   |A44  |A20  |X    |A15  |A1   |A10  |A6   |A28  |I[]  |
------------------------------------------------------------
|X    |X    |A26  |I[]  |A18  |A0   |X    |A1   |X    |A5   |
------------------------------------------------------------
|A2   |A19  |A21  |A25  |A16  |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A2   |A10  |A1   |A6   |A2   |A0   |A0   |A0   |B198 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:02<00:00, 16.27step/s]


ep_num  0
step  200
------------------------------------------------------------
|A0   |X    |X    |A0   |A87  |I[]  |A79  |A0   |X    |X    |
------------------------------------------------------------
|A4   |A15  |X    |A73  |X    |A93  |A91  |X    |A27  |A2   |
------------------------------------------------------------
|A45  |A77  |A23  |A19  |X    |A78  |A67  |A0   |A54  |A45  |
------------------------------------------------------------
|A44  |A2   |A19  |I[]  |A17  |A101 |A16  |A4   |A20  |A17  |
------------------------------------------------------------
|A0   |A88  |A30  |X    |A25  |A1   |A5   |A4   |A56  |I[]  |
------------------------------------------------------------
|X    |X    |A30  |I[]  |A32  |A2   |X    |A1   |X    |A12  |
------------------------------------------------------------
|A2   |A27  |A32  |A26  |A36  |A0   |A1   |X    |A1   |A1   |
------------------------------------------------------------
|A1   |A29  |A1   |A0   |A0   |A1   |A2   |A0   |B427 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B1592|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:37<00:00, 12.71step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B1592|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:41<00:00, 12.38step/s]


The old_packets is  3421
Finish


In [76]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 0 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_0 = ReplayBuffer(capacity)

graph_k_0 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_0.from_file_graph(map_name)
graph_k_0.renderMap()

###################################################################################################################################

model_eval_k_0 = Final_Model(graph_k_0.n_nodes, graph_k_0.len_feature, graph_k_0.action_space)
model_eval_k_0.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_0.k) + '/results_GNN_10_10_reduce_0_step_num_300.pth'))
model_eval_k_0.eval()

device = torch.device( "cpu")
model_eval_k_0 = model_eval_k_0.to(device)


#print(model_eval_k_0) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_0.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_0.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_0) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_0, tot_packets_sent_k_0, mean_ttl_received_k_0, rec_rate_k_0 = Evaluation(graph_k_0, model_eval_k_0, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_0) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_0) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_0) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_0) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:51<00:00, 17.91step/s]

Finish


In [77]:
num_packets_received_k_0

[3421, 3421, 3421, 3421, 3421, 3421, 3421, 3421, 3421, 3421]

In [78]:
with open('results_GNN_10_10_0/k_' + str(graph_k_0.k) + '/num_packets_received_k_0.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_0, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_0.k) + '/tot_packets_sent_k_0.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_0, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_0.k) + '/mean_ttl_received_k_0.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_0, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_0.k) + '/rec_rate_k_0.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_0, file)

## k = 1

In [77]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 1 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3 

capacity = 200000
buff_k_1 = ReplayBuffer(capacity)

graph_k_1 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_1.from_file_graph(map_name)
graph_k_1.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_1 = Final_Model(graph_k_1.n_nodes, graph_k_1.len_feature, graph_k_1.action_space) # model
model_t_k_1 = Final_Model(graph_k_1.n_nodes, graph_k_1.len_feature, graph_k_1.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_1.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_1 = model_k_1.to(device)
model_t_k_1 = model_t_k_1.to(device)

print(model_k_1) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_1.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_1.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_1) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_1, model_k_1, model_t_k_1, optimizer, criterion, device, buff_k_1, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:36<00:00, 12.77step/s]


ep_num  0
step  100
------------------------------------------------------------
|A1   |X    |X    |A0   |A36  |I[]  |A29  |A19  |X    |X    |
------------------------------------------------------------
|A0   |A16  |X    |A33  |X    |A34  |A48  |X    |A2   |A1   |
------------------------------------------------------------
|A0   |A38  |A19  |A37  |X    |A3   |A17  |A0   |A10  |A11  |
------------------------------------------------------------
|A2   |A72  |A15  |I[]  |A21  |A54  |A17  |A32  |A0   |A0   |
------------------------------------------------------------
|A25  |A45  |A12  |X    |A19  |A7   |A1   |A0   |A4   |I[]  |
------------------------------------------------------------
|X    |X    |A4   |I[]  |A9   |A1   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A2   |A2   |A4   |A1   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A4   |A6   |A0   |A6   |A1   |A0   |A0   |B248 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:33<00:00, 13.03step/s]


ep_num  0
step  200
------------------------------------------------------------
|A1   |X    |X    |A4   |A86  |I[]  |A76  |A35  |X    |X    |
------------------------------------------------------------
|A3   |A48  |X    |A61  |X    |A84  |A87  |X    |A1   |A0   |
------------------------------------------------------------
|A8   |A67  |A79  |A42  |X    |A4   |A25  |A1   |A38  |A23  |
------------------------------------------------------------
|A9   |A135 |A31  |I[]  |A43  |A82  |A21  |A65  |A1   |A0   |
------------------------------------------------------------
|A54  |A103 |A35  |X    |A36  |A4   |A7   |A0   |A2   |I[]  |
------------------------------------------------------------
|X    |X    |A4   |I[]  |A10  |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A4   |A5   |A7   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A17  |A10  |A4   |A5   |A0   |A2   |A0   |B483 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B2013|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:46<00:00, 11.98step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B2013|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:47<00:00, 11.97step/s]


The old_packets is  3213
Finish


In [79]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 1 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_1 = ReplayBuffer(capacity)

graph_k_1 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_1.from_file_graph(map_name)
graph_k_1.renderMap()

###################################################################################################################################

model_eval_k_1 = Final_Model(graph_k_1.n_nodes, graph_k_1.len_feature, graph_k_1.action_space)
model_eval_k_1.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_1.k) + '/results_GNN_10_10_reduce_1_step_num_300.pth'))
model_eval_k_1.eval()

device = torch.device( "cpu")
model_eval_k_1 = model_eval_k_1.to(device)


#print(model_eval_k_1) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_1.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_1.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_1) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_1, tot_packets_sent_k_1, mean_ttl_received_k_1, rec_rate_k_1 = Evaluation(graph_k_1, model_eval_k_1, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_1) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_1) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_1) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_1) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:52<00:00, 17.72step/s]

Finish


In [80]:
num_packets_received_k_1

[3213, 3213, 3213, 3213, 3213, 3213, 3213, 3213, 3213, 3213]

In [81]:
with open('results_GNN_10_10_0/k_' + str(graph_k_1.k) + '/num_packets_received_k_1.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_1, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_1.k) + '/tot_packets_sent_k_1.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_1, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_1.k) + '/mean_ttl_received_k_1.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_1, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_1.k) + '/rec_rate_k_1.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_1, file)

## k = 2

In [76]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 2 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3 

capacity = 200000
buff_k_2 = ReplayBuffer(capacity)

graph_k_2 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_2.from_file_graph(map_name)
graph_k_2.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_2 = Final_Model(graph_k_2.n_nodes, graph_k_2.len_feature, graph_k_2.action_space) # model
model_t_k_2 = Final_Model(graph_k_2.n_nodes, graph_k_2.len_feature, graph_k_2.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_2.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_2 = model_k_2.to(device)
model_t_k_2 = model_t_k_2.to(device)

print(model_k_2) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_2.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_2.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_2) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_2, model_k_2, model_t_k_2, optimizer, criterion, device, buff_k_2, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:40<00:00, 19.97step/s]


ep_num  0
step  100
------------------------------------------------------------
|A1   |X    |X    |A1   |A0   |I[]  |A2   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A27  |X    |A10  |A1   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A1   |A12  |A22  |X    |A12  |A22  |A1   |A2   |A0   |
------------------------------------------------------------
|A0   |A0   |A63  |I[]  |A13  |A14  |A90  |A57  |A16  |A8   |
------------------------------------------------------------
|A0   |A0   |A8   |X    |A11  |A6   |A100 |A28  |A36  |I[]  |
------------------------------------------------------------
|X    |X    |A6   |I[]  |A9   |A9   |X    |A18  |X    |A9   |
------------------------------------------------------------
|A0   |A0   |A15  |A17  |A33  |A8   |A19  |X    |A5   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A1   |A3   |A2   |A0   |A0   |B281 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:43<00:00, 19.27step/s]


ep_num  0
step  200
------------------------------------------------------------
|A0   |X    |X    |A0   |A7   |I[]  |A11  |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A88  |X    |A15  |A20  |X    |A1   |A0   |
------------------------------------------------------------
|A0   |A1   |A18  |A18  |X    |A79  |A31  |A5   |A5   |A1   |
------------------------------------------------------------
|A0   |A1   |A111 |I[]  |A13  |A84  |A184 |A102 |A14  |A10  |
------------------------------------------------------------
|A0   |A6   |A20  |X    |A13  |A1   |A154 |A21  |A85  |I[]  |
------------------------------------------------------------
|X    |X    |A12  |I[]  |A14  |A26  |X    |A98  |X    |A7   |
------------------------------------------------------------
|A0   |A0   |A11  |A18  |A44  |A7   |A11  |X    |A7   |A0   |
------------------------------------------------------------
|A0   |A1   |A1   |A0   |A0   |A2   |A0   |A0   |B614 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B2000|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:15<00:00, 14.73step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B2000|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:22<00:00, 14.00step/s]


The old_packets is  3322
Finish


In [82]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 2 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_2 = ReplayBuffer(capacity)

graph_k_2 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_2.from_file_graph(map_name)
graph_k_2.renderMap()

###################################################################################################################################

model_eval_k_2 = Final_Model(graph_k_2.n_nodes, graph_k_2.len_feature, graph_k_2.action_space)
model_eval_k_2.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_2.k) + '/results_GNN_10_10_reduce_2_step_num_400.pth'))
model_eval_k_2.eval()

device = torch.device( "cpu")
model_eval_k_2 = model_eval_k_2.to(device)


#print(model_eval_k_2) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_2.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_2.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_2) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_2, tot_packets_sent_k_2, mean_ttl_received_k_2, rec_rate_k_2 = Evaluation(graph_k_2, model_eval_k_2, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_2) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_2) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_2) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_2) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:50<00:00, 18.03step/s]

Finish


In [83]:
num_packets_received_k_2

[3322, 3322, 3322, 3322, 3322, 3322, 3322, 3322, 3322, 3322]

In [84]:
with open('results_GNN_10_10_0/k_' + str(graph_k_2.k) + '/num_packets_received_k_2.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_2, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_2.k) + '/tot_packets_sent_k_2.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_2, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_2.k) + '/mean_ttl_received_k_2.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_2, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_2.k) + '/rec_rate_k_2.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_2, file)

## k = 3

In [77]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 3 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3 

capacity = 200000
buff_k_3 = ReplayBuffer(capacity)

graph_k_3 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_3.from_file_graph(map_name)
graph_k_3.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_3 = Final_Model(graph_k_3.n_nodes, graph_k_3.len_feature, graph_k_3.action_space) # model
model_t_k_3 = Final_Model(graph_k_3.n_nodes, graph_k_3.len_feature, graph_k_3.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_3.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_3 = model_k_3.to(device)
model_t_k_3 = model_t_k_3.to(device)

print(model_k_3) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_3.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_3.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_3) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_3, model_k_3, model_t_k_3, optimizer, criterion, device, buff_k_3, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:23<00:00, 13.96step/s]


ep_num  0
step  100
------------------------------------------------------------
|A0   |X    |X    |A3   |A28  |I[]  |A25  |A15  |X    |X    |
------------------------------------------------------------
|A0   |A1   |X    |A4   |X    |A28  |A30  |X    |A0   |A0   |
------------------------------------------------------------
|A1   |A0   |A46  |A47  |X    |A1   |A22  |A9   |A7   |A1   |
------------------------------------------------------------
|A1   |A52  |A28  |I[]  |A26  |A11  |A0   |A1   |A12  |A7   |
------------------------------------------------------------
|A2   |A10  |A20  |X    |A28  |A0   |A1   |A0   |A10  |I[]  |
------------------------------------------------------------
|X    |X    |A6   |I[]  |A10  |A0   |X    |A1   |X    |A0   |
------------------------------------------------------------
|A0   |A2   |A4   |A3   |A1   |A0   |A1   |X    |A3   |A1   |
------------------------------------------------------------
|A0   |A11  |A19  |A23  |A2   |A1   |A0   |A3   |B424 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:21<00:00, 14.16step/s]


ep_num  0
step  200
------------------------------------------------------------
|A0   |X    |X    |A2   |A56  |I[]  |A60  |A36  |X    |X    |
------------------------------------------------------------
|A0   |A1   |X    |A8   |X    |A58  |A59  |X    |A1   |A0   |
------------------------------------------------------------
|A2   |A7   |A84  |A48  |X    |A14  |A27  |A6   |A19  |A2   |
------------------------------------------------------------
|A10  |A120 |A40  |I[]  |A50  |A39  |A10  |A9   |A18  |A12  |
------------------------------------------------------------
|A23  |A57  |A37  |X    |A45  |A15  |A6   |A0   |A21  |I[]  |
------------------------------------------------------------
|X    |X    |A38  |I[]  |A13  |A4   |X    |A2   |X    |A12  |
------------------------------------------------------------
|A0   |A0   |A11  |A7   |A5   |A0   |A0   |X    |A3   |A0   |
------------------------------------------------------------
|A0   |A0   |A6   |A21  |A2   |A0   |A3   |A2   |B866 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3199|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:45<00:00, 12.10step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3199|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:41<00:00, 12.41step/s]

The old_packets is  3788
Finish


In [85]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 3 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_3 = ReplayBuffer(capacity)

graph_k_3 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_3.from_file_graph(map_name)
graph_k_3.renderMap()

###################################################################################################################################

model_eval_k_3 = Final_Model(graph_k_3.n_nodes, graph_k_3.len_feature, graph_k_3.action_space)
model_eval_k_3.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_3.k) + '/results_GNN_10_10_reduce_3_step_num_200.pth'))
model_eval_k_3.eval()

device = torch.device( "cpu")
model_eval_k_3 = model_eval_k_3.to(device)


#print(model_eval_k_3) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_3.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_3.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_3) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_3, tot_packets_sent_k_3, mean_ttl_received_k_3, rec_rate_k_3 = Evaluation(graph_k_3, model_eval_k_3, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_3) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_3) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_3) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_3) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:44<00:00, 19.22step/s]

Finish


In [86]:
num_packets_received_k_3

[3788, 3788, 3788, 3788, 3788, 3788, 3788, 3788, 3788, 3788]

In [87]:
with open('results_GNN_10_10_0/k_' + str(graph_k_3.k) + '/num_packets_received_k_3.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_3, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_3.k) + '/tot_packets_sent_k_3.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_3, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_3.k) + '/mean_ttl_received_k_3.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_3, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_3.k) + '/rec_rate_k_3.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_3, file)

## k = 4

In [76]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 4 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3 

capacity = 200000
buff_k_4 = ReplayBuffer(capacity)

graph_k_4 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_4.from_file_graph(map_name)
graph_k_4.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_4 = Final_Model(graph_k_4.n_nodes, graph_k_4.len_feature, graph_k_4.action_space) # model
model_t_k_4 = Final_Model(graph_k_4.n_nodes, graph_k_4.len_feature, graph_k_4.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_4.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_4 = model_k_4.to(device)
model_t_k_4 = model_t_k_4.to(device)

print(model_k_4) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_4.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_4.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_4) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_4, model_k_4, model_t_k_4, optimizer, criterion, device, buff_k_4, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:32<00:00, 21.58step/s]


ep_num  0
step  100
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A9   |A3   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A6   |X    |A0   |A7   |A2   |A1   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A2   |A3   |A33  |A61  |A31  |A27  |
------------------------------------------------------------
|A0   |A0   |A1   |X    |A0   |A3   |A89  |A19  |A82  |I[]  |
------------------------------------------------------------
|X    |X    |A9   |I[]  |A2   |A41  |X    |A0   |X    |A29  |
------------------------------------------------------------
|A0   |A1   |A0   |A6   |A12  |A0   |A1   |X    |A19  |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A1   |A23  |A35  |A20  |A0   |B413 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:39<00:00, 20.04step/s]


ep_num  0
step  200
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A7   |A4   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A4   |A30  |X    |A1   |A52  |A0   |A1   |A0   |
------------------------------------------------------------
|A0   |A0   |A15  |I[]  |A5   |A8   |A63  |A119 |A42  |A45  |
------------------------------------------------------------
|A0   |A0   |A17  |X    |A15  |A0   |A137 |A32  |A223 |I[]  |
------------------------------------------------------------
|X    |X    |A22  |I[]  |A16  |A44  |X    |A5   |X    |A45  |
------------------------------------------------------------
|A0   |A0   |A10  |A47  |A37  |A0   |A1   |X    |A16  |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A80  |A46  |A0   |B796 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3062|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:10<00:00, 15.27step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3062|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:11<00:00, 15.26step/s]


The old_packets is  3428
Finish


In [88]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 4 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_4 = ReplayBuffer(capacity)

graph_k_4 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_4.from_file_graph(map_name)
graph_k_4.renderMap()

###################################################################################################################################

model_eval_k_4 = Final_Model(graph_k_4.n_nodes, graph_k_4.len_feature, graph_k_4.action_space)
model_eval_k_4.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_4.k) + '/results_GNN_10_10_reduce_4_step_num_1500.pth'))
model_eval_k_4.eval()

device = torch.device( "cpu")
model_eval_k_4 = model_eval_k_4.to(device)


#print(model_eval_k_4) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_4.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_4.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_4) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_4, tot_packets_sent_k_4, mean_ttl_received_k_4, rec_rate_k_4 = Evaluation(graph_k_4, model_eval_k_4, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_4) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_4) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_4) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_4) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:38<00:00, 20.40step/s]

Finish


In [89]:
num_packets_received_k_4

[3428, 3428, 3428, 3428, 3428, 3428, 3428, 3428, 3428, 3428]

In [90]:
with open('results_GNN_10_10_0/k_' + str(graph_k_4.k) + '/num_packets_received_k_4.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_4, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_4.k) + '/tot_packets_sent_k_4.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_4, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_4.k) + '/mean_ttl_received_k_4.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_4, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_4.k) + '/rec_rate_k_4.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_4, file)

## k = 5

In [77]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 5 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3 

capacity = 200000
buff_k_5 = ReplayBuffer(capacity)

graph_k_5 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_5.from_file_graph(map_name)
graph_k_5.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_5 = Final_Model(graph_k_5.n_nodes, graph_k_5.len_feature, graph_k_5.action_space) # model
model_t_k_5 = Final_Model(graph_k_5.n_nodes, graph_k_5.len_feature, graph_k_5.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_5.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_5 = model_k_5.to(device)
model_t_k_5 = model_t_k_5.to(device)

print(model_k_5) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_5.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_5.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_5) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_5, model_k_5, model_t_k_5, optimizer, criterion, device, buff_k_5, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:05<00:00, 15.93step/s]


ep_num  0
step  100
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A4   |A1   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A4   |A10  |X    |A0   |A2   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A44  |I[]  |A5   |A1   |A31  |A29  |A0   |A1   |
------------------------------------------------------------
|A0   |A1   |A6   |X    |A2   |A38  |A143 |A34  |A61  |I[]  |
------------------------------------------------------------
|X    |X    |A8   |I[]  |A2   |A9   |X    |A0   |X    |A5   |
------------------------------------------------------------
|A0   |A0   |A10  |A12  |A24  |A19  |A0   |X    |A0   |A1   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A1   |A34  |A56  |A0   |B388 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:01<00:00, 16.46step/s]


ep_num  0
step  200
------------------------------------------------------------
|A0   |X    |X    |A0   |A1   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A2   |X    |A6   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A8   |A18  |X    |A2   |A1   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A90  |I[]  |A7   |A23  |A98  |A157 |A4   |A0   |
------------------------------------------------------------
|A0   |A0   |A13  |X    |A9   |A54  |A290 |A43  |A105 |I[]  |
------------------------------------------------------------
|X    |X    |A15  |I[]  |A14  |A70  |X    |A2   |X    |A5   |
------------------------------------------------------------
|A0   |A0   |A17  |A17  |A39  |A3   |A1   |X    |A1   |A1   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A10  |A88  |A0   |B750 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3249|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:10<00:00, 15.38step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3249|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:12<00:00, 15.15step/s]


The old_packets is  4000
Finish


In [91]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 5 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_5 = ReplayBuffer(capacity)

graph_k_5 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_5.from_file_graph(map_name)
graph_k_5.renderMap()

###################################################################################################################################

model_eval_k_5 = Final_Model(graph_k_5.n_nodes, graph_k_5.len_feature, graph_k_5.action_space)
model_eval_k_5.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_5.k) + '/results_GNN_10_10_reduce_5_step_num_600.pth'))
model_eval_k_5.eval()

device = torch.device( "cpu")
model_eval_k_5 = model_eval_k_5.to(device)


#print(model_eval_k_5) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_5.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_5.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_5) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_5, tot_packets_sent_k_5, mean_ttl_received_k_5, rec_rate_k_5 = Evaluation(graph_k_5, model_eval_k_5, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_5) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_5) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_5) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_5) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:35<00:00, 20.99step/s]

Finish


In [92]:
num_packets_received_k_5

[4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000, 4000]

In [93]:
with open('results_GNN_10_10_0/k_' + str(graph_k_5.k) + '/num_packets_received_k_5.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_5, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_5.k) + '/tot_packets_sent_k_5.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_5, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_5.k) + '/mean_ttl_received_k_5.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_5, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_5.k) + '/rec_rate_k_5.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_5, file)

## k = 6

In [76]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 6 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 # 15 #10
epsilon = 0.3 

capacity = 200000
buff_k_6 = ReplayBuffer(capacity)

graph_k_6 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_6.from_file_graph(map_name)
graph_k_6.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_6 = Final_Model(graph_k_6.n_nodes, graph_k_6.len_feature, graph_k_6.action_space) # model
model_t_k_6 = Final_Model(graph_k_6.n_nodes, graph_k_6.len_feature, graph_k_6.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_6.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_6 = model_k_6.to(device)
model_t_k_6 = model_t_k_6.to(device)

print(model_k_6) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_6.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_6.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_6) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_6, model_k_6, model_t_k_6, optimizer, criterion, device, buff_k_6, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:42<00:00, 19.48step/s]


ep_num  0
step  100
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A3   |X    |A0   |A2   |A1   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A3   |A0   |A2   |A14  |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A2   |X    |A0   |A1   |A2   |A1   |A138 |I[]  |
------------------------------------------------------------
|X    |X    |A4   |I[]  |A0   |A49  |X    |A55  |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A4   |A0   |A2   |A23  |X    |A73  |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A1   |A24  |A48  |A57  |A70  |B420 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:48<00:00, 18.48step/s]


ep_num  0
step  200
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A1   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A14  |X    |A0   |A3   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A4   |I[]  |A3   |A0   |A3   |A1   |A2   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A5   |A0   |A1   |A0   |A281 |I[]  |
------------------------------------------------------------
|X    |X    |A2   |I[]  |A0   |A77  |X    |A147 |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A1   |A5   |A6   |A3   |A51  |X    |A131 |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A46  |A96  |A135 |A142 |B838 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3738|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [02:04<00:00, 16.02step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3738|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:59<00:00, 16.72step/s]


The old_packets is  3600
Finish


In [94]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 6 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_6 = ReplayBuffer(capacity)

graph_k_6 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_6.from_file_graph(map_name)
graph_k_6.renderMap()

###################################################################################################################################

model_eval_k_6 = Final_Model(graph_k_6.n_nodes, graph_k_6.len_feature, graph_k_6.action_space)
model_eval_k_6.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_6.k) + '/results_GNN_10_10_reduce_6_step_num_300.pth'))
model_eval_k_6.eval()

device = torch.device( "cpu")
model_eval_k_6 = model_eval_k_6.to(device)


#print(model_eval_k_6) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_6.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_6.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_6) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_6, tot_packets_sent_k_6, mean_ttl_received_k_6, rec_rate_k_6 = Evaluation(graph_k_6, model_eval_k_6, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_6) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_6) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_6) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_6) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:34<00:00, 21.16step/s]

Finish


In [95]:
num_packets_received_k_6

[3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600, 3600]

In [96]:
with open('results_GNN_10_10_0/k_' + str(graph_k_6.k) + '/num_packets_received_k_6.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_6, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_6.k) + '/tot_packets_sent_k_6.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_6, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_6.k) + '/mean_ttl_received_k_6.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_6, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_6.k) + '/rec_rate_k_6.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_6, file)

## k = 7

In [77]:
# Training
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 7 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3 

capacity = 200000
buff_k_7 = ReplayBuffer(capacity)

graph_k_7 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_7.from_file_graph(map_name)
graph_k_7.renderMap()

###################################################################################################################################
device = torch.device("cpu")

model_k_7 = Final_Model(graph_k_7.n_nodes, graph_k_7.len_feature, graph_k_7.action_space) # model
model_t_k_7 = Final_Model(graph_k_7.n_nodes, graph_k_7.len_feature, graph_k_7.action_space) # target model

EP_NUM = 0
old_packets = 0

optimizer = optim.Adam(model_k_7.parameters(), lr=0.001)
criterion = nn.MSELoss()

model_k_7 = model_k_7.to(device)
model_t_k_7 = model_t_k_7.to(device)

print(model_k_7) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

batch_size = 8 #10
num_ep = 1 #5

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_7.k)

log_file = open(folder_name + '/DGN_train_log_10_10.txt','a')
log_file_eval = open(folder_name + '/DGN_train_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_7.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_k_7) # Print model architecture

sys.stdout = orig_stdout
log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('EP_NUM' + ' ' + str(EP_NUM) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
memory_start = psutil.virtual_memory().used

num_packets_received, tot_packets_sent, mean_ttl_received, rec_rate, old_packets = Training(graph_k_7, model_k_7, model_t_k_7, optimizer, criterion, device, buff_k_7, batch_size, log_file, log_file_eval, folder_name, num_ep, num_steps, iot_generate_time, create_packets_till, EP_NUM=EP_NUM, old_packets=old_packets)

print("The old_packets is ", old_packets)
finish_time = time.time()
memory_end = psutil.virtual_memory().used

memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################
log_file.close()
print("Finish")

###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:57<00:00, 17.08step/s]


ep_num  0
step  100
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A1   |A1   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A1   |A0   |A0   |A1   |A10  |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A1   |A0   |A0   |A195 |I[]  |
------------------------------------------------------------
|X    |X    |A4   |I[]  |A0   |A49  |X    |A1   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A5   |A0   |A0   |A1   |X    |A92  |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A48  |A46  |A49  |A99  |B396 |A0 

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:46<00:00, 18.72step/s]


ep_num  0
step  200
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A1   |A1   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A1   |A0   |A0   |A1   |A10  |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A1   |A0   |A0   |A395 |I[]  |
------------------------------------------------------------
|X    |X    |A4   |I[]  |A0   |A99  |X    |A1   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A5   |A0   |A0   |A1   |X    |A192 |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A98  |A96  |A99  |A199 |B796 |A0 

...

ep_num  0
step  1900
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3574|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:52<00:00, 17.82step/s]


ep_num  0
step  1999
------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B3574|A0

Episode 0, Number of Steps : 100%|████████| 2000/2000 [01:50<00:00, 18.04step/s]


The old_packets is  3574
Finish


In [97]:
# Evaluation

seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

###################################################################################################################################

n = 10
m = 10
k = 7 # Number of neighbours to omit
neighbors = 8

def_ttl = 1000
iot_rate = 20 #10
epsilon = 0.3

capacity = 200000
buff_k_7 = ReplayBuffer(capacity)

graph_k_7 = Graph(n,m,k,neighbors, iot_rate, def_ttl, epsilon)
map_name = "Maps/maps_10_10/map_10_10_0_8e.pkl"
graph_k_7.from_file_graph(map_name)
graph_k_7.renderMap()

###################################################################################################################################

model_eval_k_7 = Final_Model(graph_k_7.n_nodes, graph_k_7.len_feature, graph_k_7.action_space)
model_eval_k_7.load_state_dict(torch.load('results_GNN_10_10_0/k_' + str(graph_k_7.k) + '/results_GNN_10_10_reduce_7_step_num_0.pth'))
model_eval_k_7.eval()

device = torch.device( "cpu")
model_eval_k_7 = model_eval_k_7.to(device)


#print(model_eval_k_7) # Print model architecture

###################################################################################################################################

num_steps = 2000
iot_generate_time = 10
create_packets_till = 400 #100


###################################################################################################################################

num_ep = 10

###################################################################################################################################

folder_name = 'results_GNN_10_10_0/k_' + str(graph_k_7.k) + '/evaluation'
log_file = open(folder_name + '/DGN_eval_log_10_10.txt','a')

log_file.write('\n\n\n')
log_file.write('Set Reproducibility' + '\n\n\n')

log_file.write('seed = 0' + '\n')
log_file.write('random.seed(seed)' + '\n')
log_file.write('torch.manual_seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed(seed)' + '\n')
log_file.write('np.random.seed(seed)' + '\n')
log_file.write('torch.cuda.manual_seed_all(seed)' + '\n')
log_file.write('os.environ[\'PYTHONHASHSEED\'] = str(seed)' + '\n')
log_file.write('os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"' + '\n')

log_file.write('torch.backends.cudnn.deterministic = True' + '\n')
log_file.write('torch.backends.cudnn.benchmark = False' + '\n')
log_file.write('torch.use_deterministic_algorithms(True)' + '\n')
log_file.write('2 hop network' + '\n')

log_file.write('\n\n\n')
log_file.write('Model Architecture' + '\n\n\n')

log_file.write("lr=0.001" + '\n')
log_file.write("optimizer = optim.Adam(model_k_7.parameters(), lr=0.001)" + "\n")
log_file.write("criterion = nn.MSELoss()" + "\n")


orig_stdout = sys.stdout
sys.stdout = log_file

print(model_eval_k_7) # Print model architecture

sys.stdout = orig_stdout

log_file.write('\n\n\n')
log_file.write('Model Hyperparameters' + '\n\n\n')

log_file.write('n' + ' ' + str(n) + '\n')
log_file.write('m' + ' ' + str(m) + '\n')
log_file.write('k' + ' ' + str(k) + '\n')
log_file.write('neighbors' + ' ' + str(neighbors) + '\n')
log_file.write('def_ttl' + ' ' + str(def_ttl) + '\n')
log_file.write('iot_rate' + ' ' + str(iot_rate) + '\n')
log_file.write('epsilon' + ' ' + str(epsilon) + '\n')
log_file.write('capacity' + ' ' + str(capacity) + '\n')
log_file.write('num_steps' + ' ' + str(num_steps) + '\n')
log_file.write('iot_generate_time' + ' ' + str(iot_generate_time) + '\n')
log_file.write('create_packets_till' + ' ' + str(create_packets_till) + '\n')
#log_file.write('batch_size' + ' ' + str(batch_size) + '\n')
log_file.write('num_ep' + ' ' + str(num_ep) + '\n')

log_file.write('\n\n\n')
log_file.write('Model Training' + '\n\n\n')


start_time = time.time()
#cpu_percent_start = psutil.cpu_percent(interval=None)
memory_start = psutil.virtual_memory().used

num_packets_received_k_7, tot_packets_sent_k_7, mean_ttl_received_k_7, rec_rate_k_7 = Evaluation(graph_k_7, model_eval_k_7, device, log_file, num_ep, num_steps, iot_generate_time, create_packets_till)
finish_time = time.time()
#cpu_percent_end = psutil.cpu_percent(interval=None)
memory_end = psutil.virtual_memory().used

#cpu_usage = cpu_percent_end - cpu_percent_start
memory_usage = (memory_end - memory_start) >> 20 # in MB
duration = finish_time - start_time

log_file.write('\n\n\n')

log_file.write('num_packets_received' + ' ' + str(num_packets_received_k_7) + '\n')
log_file.write('tot_packets_sent' + ' ' + str(tot_packets_sent_k_7) + '\n')
log_file.write('mean_ttl_received' + ' ' + str(mean_ttl_received_k_7) + '\n')
log_file.write('rec_rate' + ' ' + str(rec_rate_k_7) + '\n')

log_file.write('Evaluation Metrics' + '\n\n\n')
log_file.write('Duration ' + ' ' + str(duration) + '\n')
#log_file.write('CPU Usage (in %)' + ' ' + str(cpu_usage) + '\n')
log_file.write('RAM Memory Usage (in MB) ' + ' ' + str(memory_usage) + '\n')


###################################################################################################################################


log_file.close()
print("Finish")
###################################################################################################################################


------------------------------------------------------------
|A0   |X    |X    |A0   |A0   |I[]  |A0   |A0   |X    |X    |
------------------------------------------------------------
|A0   |A0   |X    |A0   |X    |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |I[]  |A0   |A0   |A0   |A0   |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |X    |A0   |A0   |A0   |A0   |A0   |I[]  |
------------------------------------------------------------
|X    |X    |A0   |I[]  |A0   |A0   |X    |A0   |X    |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |X    |A0   |A0   |
------------------------------------------------------------
|A0   |A0   |A0   |A0   |A0   |A0   |A0   |A0   |B0   |A0   |
----------------

Episode 9, Number of Steps : 100%|████████| 2000/2000 [01:30<00:00, 22.09step/s]

Finish


In [98]:
num_packets_received_k_7

[3574, 3574, 3574, 3574, 3574, 3574, 3574, 3574, 3574, 3574]

In [99]:
with open('results_GNN_10_10_0/k_' + str(graph_k_7.k) + '/num_packets_received_k_7.pkl', 'wb') as file:
    pickle.dump(num_packets_received_k_7, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_7.k) + '/tot_packets_sent_k_7.pkl', 'wb') as file:
    pickle.dump(tot_packets_sent_k_7, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_7.k) + '/mean_ttl_received_k_7.pkl', 'wb') as file:
    pickle.dump(mean_ttl_received_k_7, file)
with open('results_GNN_10_10_0/k_' + str(graph_k_7.k) + '/rec_rate_k_7.pkl', 'wb') as file:
    pickle.dump(rec_rate_k_7, file)